<a href="https://colab.research.google.com/github/shake/colab-Llama-2-ipynb/blob/main/Llama_2%E5%8E%9F%E5%A7%8B%E6%A8%A1%E5%9E%8B%E8%BD%AC%E6%8D%A2HF%E6%A0%BC%E5%BC%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

原始模型的转换，这个我对着文档操作过无数次，已经很熟练，这次计划把格式转换的模型，上传到hugingface上，日后对模型是微调，就从自己转换的HF格式，开始进行微调。

另外一个选择，就是直接使用Facebook提供的HF格式。

**重点**

* colab，你需要High-RAM才能完成转换。
* 只需要修改**MODEL_ID** 就可以，支持Llama-2 所有版本。
* colab保存huggingface的token，如果需要上传，需要写入权限的token
* 上传转换的模型到hugingface
* 7B是13G，13B是26G，70B是140G。进行转换，需要磁盘至少有3倍的剩余空间。

In [1]:
# 输入你需要转换的Llama版本，username 是你登录hugingface的用户名
MODEL_ID = "meta-llama/Llama-2-7b"
MODEL_NAME = MODEL_ID.split('/')[-1]
NEW_MODEL = MODEL_NAME+'-hf'
Parameter = str.upper(MODEL_NAME.split('-')[2])


In [2]:
print(Parameter)

7B


In [ ]:
# 安装需要依赖
%%capture
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2  trl==0.4.7
!pip install -q huggingface_hub sentencepiece

In [ ]:
# import 密钥，token
%%capture
from google.colab import userdata
hf_token = userdata.get('huggingface')
!git config --global credential.helper store
!huggingface-cli login --token $hf_token --add-to-git-credential

In [7]:
import huggingface_hub
username = f'{huggingface_hub.whoami()["name"]}'
print(username)

chenshake


In [ ]:
# 可以整个目录进行下载，这里专门单独选择文件来下载 ,去掉--quiet参数，可以看到详细的下载过程。
!huggingface-cli download --quiet\
	--local-dir=./$MODEL_NAME \
	$MODEL_ID \
  --include "*.json"   "*.model" "*.chk" "*.pth"

In [ ]:
# 查看下载结果,文件是下载到cache，做了一个链接到目录下，所以无法直接查看文件大小。
import os.path
from pathlib import Path
path=os.path.join(MODEL_NAME)
print(path)
!ls -lsh ./$MODEL_NAME

In [ ]:
# 格式转换需要提前创建好目录
!mkdir ./$MODEL_NAME/$Parameter
!cp ./$MODEL_NAME/params.json ./$MODEL_NAME/$Parameter/params.json

In [ ]:
!ls ./$MODEL_NAME/$Parameter

In [ ]:
# update cache，有时候会出现一个transformers的 cache 的错误提示，提前运行，解决这个问题。
from transformers.utils.hub import move_cache

In [ ]:
#解决colab字符集错误，遇到错误再运行也是可以。
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
# 下载HF格式转换工具
!wget -q https://raw.githubusercontent.com/huggingface/transformers/main/src/transformers/models/llama/convert_llama_weights_to_hf.py

In [ ]:
# 开始转换, 消耗内存超过16G，默认内存，会导致中断。
!python convert_llama_weights_to_hf.py --model_size $Parameter \
    --input_dir ./$MODEL_NAME \
    --output_dir ./$NEW_MODEL

In [ ]:
# 调用api，在你的hugingface账号下，创建repo，存放模型，记得修改上面的用户名的变量。
from huggingface_hub import create_repo, HfApi

# Create empty repo
create_repo(
    repo_id=f"{username}/{NEW_MODEL}",
    repo_type="model",
    exist_ok=True,
)

In [ ]:
# 把转换HF格式，上传到huggingface,你账号的model里。
api = HfApi()
api.upload_folder(
	folder_path=f"{NEW_MODEL}",
	repo_id=f"{username}/{NEW_MODEL}",
	repo_type="model",
)

